In [24]:
from ultralytics import YOLO
import cv2
import threading
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [18]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

In [26]:
pygame.mixer.init()

def play_sound(sound_path):
    pygame.mixer.music.load(sound_path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():  # Wait for audio to finish playing
        if not is_video_playing:
            pygame.mixer.music.stop()
            
is_video_playing = True

def define_restricted_area(frame_width, frame_height):
    top_left_x = int(frame_width * 0.75)  # Start 75% from the left
    top_left_y = 0                        # Start from the top
    bottom_right_x = frame_width          # End at the far right
    bottom_right_y = frame_height         # End at the bottom
    return (top_left_x, top_left_y), (bottom_right_x, bottom_right_y)

def is_inside_restricted_area(x, y, top_left, bottom_right):
    if top_left[0] <= x <= bottom_right[0] and top_left[1] <= y <= bottom_right[1]:
        return True
    return False

def raise_alarm():
    print("Alarm! Person detected in restricted area.")
    # Start a new thread to play the alarm sound
    threading.Thread(target=play_sound, args=(r"C:\Users\Hamza\Desktop\Deep Learning\Assignment 3\emergency-alarm.mp3",), daemon=True).start()

# Open the video file
video_path = r"C:\Users\Hamza\Desktop\Deep Learning\Assignment 3\Video.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    success, frame = cap.read()

    if success:
        frame_height, frame_width = frame.shape[:2]
        top_left, bottom_right = define_restricted_area(frame_width, frame_height)

        # Draw the restricted area in red
        cv2.rectangle(frame, top_left, bottom_right, (0, 0, 255), 2)

        # Run YOLOv8 inference on the frame
        results = model.predict(frame, stream=True)

        for result in results:
            boxes = result.boxes.cpu().numpy()  # Get boxes
            for box in boxes:
                if (box.conf[0] > 0.5):  # Check if confidence is greater than 0.5
                    r = box.xyxy[0].astype(int)
                    class_name = result.names[int(box.cls[0])]

                    if class_name == 'person':
                        center_x = int((r[0] + r[2]) / 2)
                        center_y = int((r[1] + r[3]) / 2)

                        if is_inside_restricted_area(center_x, center_y, top_left, bottom_right):
                            raise_alarm()

                        # Draw bounding box and label for person
                        cv2.rectangle(frame, r[:2], r[2:], (255, 255, 255), 2)
                        label = f"{class_name}: {box.conf[0]:.2f}"
                        cv2.putText(frame, label, (r[0], r[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Display the frame
        cv2.imshow("YOLOv8 Inference", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

is_video_playing = False

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 2 ties, 1 chair, 1 dining table, 1 tv, 102.0ms
Speed: 2.0ms preprocess, 102.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tie, 1 chair, 1 dining table, 1 tv, 120.9ms
Speed: 3.0ms preprocess, 120.9ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 1 dining table, 1 tv, 104.3ms
Speed: 1.0ms preprocess, 104.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 1 dining table, 1 tv, 96.4ms
Speed: 1.5ms preprocess, 96.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 chairs, 1 dining table, 1 tv, 97.5ms
Speed: 1.0ms preprocess, 97.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 1 dining table, 1 tv, 91.4ms
Speed: 3.2ms preprocess, 91.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 1 di